#1. Nettoyage des données

---



In [1]:
import pandas as pd
import os

# Check if the file exists
file_path = "/content/hotels.xlsx"
if not os.path.exists(file_path):
    print(f"Error: File '{file_path}' not found.")
    df_hotels = None
else:
    try:
        xls = pd.ExcelFile(file_path)
        df_hotels = xls.parse('Hotels')
        df_comments = xls.parse('Commentaires')
        df_questions = xls.parse('QuestionReponse')
    except Exception as e:
        print(f"An error occurred while reading the Excel file: {e}")
        df_hotels = None

In [2]:
# Check data types
print(df_hotels.dtypes)

# Check for missing values
missing_values = df_hotels.isnull().sum()
missing_percentage = (missing_values / len(df_hotels)) * 100
print("\nMissing Values:\n", missing_values)
print("\nMissing Value Percentage:\n", missing_percentage)

Lieu                          object
Nom HOTEL                     object
adresse                       object
Etoile                        object
Prix                           int64
Rate nominal                  object
Rate ordinal                  object
Expériences vécues            object
points fort                   object
Lieux à proximité             object
Restaurants et cafés          object
Plages à proximité            object
Transports en commun          object
Aéroports les plus proches    object
a savoir                      object
Enfants et lits               object
Arrive                        object
depart                        object
restriction d'age             object
Animaux domestiques           object
dtype: object

Missing Values:
 Lieu                          0
Nom HOTEL                     0
adresse                       0
Etoile                        0
Prix                          0
Rate nominal                  0
Rate ordinal                  0
Expé

In [3]:
# Check data types
print(df_comments.dtypes)

# Check for missing values
missing_values = df_comments.isnull().sum()
missing_percentage = (missing_values / len(df_comments)) * 100
print("\nMissing Values:\n", missing_values)
print("\nMissing Value Percentage:\n", missing_percentage)

nom hotel           object
Note                object
Titre               object
Commentaire         object
Date commentaire    object
dtype: object

Missing Values:
 nom hotel            0
Note                17
Titre                0
Commentaire          0
Date commentaire    17
dtype: int64

Missing Value Percentage:
 nom hotel           0.000000
Note                0.458468
Titre               0.000000
Commentaire         0.000000
Date commentaire    0.458468
dtype: float64


In [4]:
# Check data types
print(df_questions.dtypes)

# Check for missing values
missing_values = df_questions.isnull().sum()
missing_percentage = (missing_values / len(df_questions)) * 100
print("\nMissing Values:\n", missing_values)
print("\nMissing Value Percentage:\n", missing_percentage)

nom hotel      object
question       object
answer_text    object
dtype: object

Missing Values:
 nom hotel      0
question       0
answer_text    0
dtype: int64

Missing Value Percentage:
 nom hotel      0.0
question       0.0
answer_text    0.0
dtype: float64


In [5]:
import re
# Function to clean text (remove special characters, accents)
def clean_text(text):
    if isinstance(text, str):
        text = text.lower()
        text = re.sub(r'[éèêë]', 'e', text)
        text = re.sub(r'[àâä]', 'a', text)
        text = re.sub(r'[îï]', 'i', text)
        text = re.sub(r'[ôö]', 'o', text)
        text = re.sub(r'[ùûü]', 'u', text)
        text = re.sub(r'[^a-zA-Z0-9\s,]', '', text)  # Remove special characters
    return text

# Cleaning Hotels sheet
df_hotels = df_hotels.rename(columns=lambda x: clean_text(x))  # Normalize column names
df_hotels['nom hotel'] = df_hotels['nom hotel'].apply(clean_text)
df_hotels['etoile'] = pd.to_numeric(df_hotels['etoile'], errors='coerce')  # Convert stars to numeric
df_hotels['prix'] = pd.to_numeric(df_hotels['prix'], errors='coerce')  # Convert price to numeric

# Cleaning Commentaires sheet
df_comments = df_comments.rename(columns=lambda x: clean_text(x))
df_comments['nom hotel'] = df_comments['nom hotel'].apply(clean_text)
df_comments['note'] = df_comments['note'].astype(str).str.replace(',', '.').astype(float)  # Convert ratings to float

# Cleaning QuestionReponse sheet
df_questions = df_questions.rename(columns=lambda x: clean_text(x))
df_questions['nom hotel'] = df_questions['nom hotel'].apply(clean_text)


## Normalize the locations ' jandouba and jendouba , delete the duplicated hotels

#3. Préparation des données pour le ChatBot

##3.1 Convertion vers Json

In [6]:
import json
import numpy as np
# Assign unique hotel IDs
df_hotels['hotel_id'] = np.arange(1, len(df_hotels) + 1)

# Merge comments and questions with hotel data
hotels_json = []
for _, hotel in df_hotels.iterrows():
    hotel_id = hotel['hotel_id']
    hotel_name = hotel['nom hotel']

    # Extract comments for the current hotel
    comments = df_comments[df_comments['nom hotel'] == hotel_name][['titre', 'commentaire', 'note', 'date commentaire']].to_dict(orient='records')

    # Extract questions & answers for the current hotel
    questions = df_questions[df_questions['nom hotel'] == hotel_name][['question', 'answertext']].to_dict(orient='records')

    # Construct hotel JSON object
    hotel_data = {
        "hotel_id": int(hotel_id),
        "name": hotel['nom hotel'],
        "location": hotel['lieu'],
        "address": hotel['adresse'],
        "stars": hotel['etoile'],
        "price": hotel['prix'],
        "rating": hotel['rate nominal'],
        "features": hotel['points fort'],
        "nearby_places": hotel['lieux a proximite'],
        "nearby_beaches": hotel['plages a proximite'],
        "transport": hotel['transports en commun'],
        "airports": hotel['aeroports les plus proches'],
        "policies": {
            "checkin": hotel['arrive'],
            "checkout": hotel['depart'],
            "age_restriction": hotel['restriction dage'],
            "pets": hotel['animaux domestiques'],
            "children_beds": hotel['enfants et lits']
        },
        "additional_info": hotel['a savoir'],
        "comments": comments,
        "faq": questions
    }

    hotels_json.append(hotel_data)

# Save to JSON file
json_output_path = "hotels_data.json"
with open(json_output_path, "w", encoding="utf-8") as json_file:
    json.dump(hotels_json, json_file, indent=4, ensure_ascii=False)

print(f"JSON file saved at: {json_output_path}")

JSON file saved at: hotels_data.json


In [ ]:
# un peu de temps pour qu'il enregistre le fichier json
import time
time.sleep(5)


# 3.2 Choix des informations

j'ai voulu travaillez avec les commentaires mais les chunks sont trés grandes et prend beaucoup de temps ( plus que 3 heures )

In [10]:
df_hotels = pd.read_json('hotels_data.json')

# nettoyage des espaces dans les caractéristiques et les plages
if df_hotels is not None and 'nearby_beaches' in df_hotels.columns:
  for index, row in df_hotels.iterrows():
    if isinstance(row['nearby_beaches'], str):
      df_hotels.at[index, 'nearby_beaches'] = row['nearby_beaches'].replace('\\n', ' ')
  print(df_hotels["nearby_beaches"])
else:
  print("DataFrame or 'nearby_beaches' column not found.")
df_hotels["nearby_beaches"]

def clean_list_field(field_value):
    try:
        items = ast.literal_eval(field_value)
        if isinstance(items, list):
            cleaned = list(dict.fromkeys([item.replace('\\n', ' ').strip().lower() for item in items if isinstance(item, str)]))
            return ', '.join(cleaned)
    except Exception:
        pass
    return 'non renseigné'

0      ['Plage de Boujaafar 500 m', 'Plage de Bhar Ez...
1      ['Plage de Bhar Ezzebla 750 m', 'Plage de Bouj...
2      ['Las Vegas Beach 850 m', 'Plage de Boujaafar ...
3      ['Plage de Boujaafar 600 m', 'Plage de Bhar Ez...
4      ['Plage de Boujaafar 700 m', 'Plage de Bhar Ez...
                             ...                        
392    ["Plages d'Hammamet 50 m", 'Plage du Sentido A...
393    ["Plages d'Hammamet 10 m", 'Plage de Yasmine H...
394    ['Plage de Mrezga 450 m', 'Plage de Hammamet 1...
395    ["Plages d'Hammamet 650 m", 'Plage de Yasmine ...
396    ['Plage de Yasmine Hammamet 550 m', "Plages d'...
Name: nearby_beaches, Length: 397, dtype: object


In [11]:
# générer une colonne qui combine tous les informations d'un seul hotel
def generate_text(row):
    name = row.get('name', 'nom inconnu')
    location = row.get('location', 'localisation inconnue')
    price = row.get('price', 'prix non renseigné')
    rating = row.get('rating', 'note non disponible')

    features = clean_list_field(row.get('features', ''))
    beaches = clean_list_field(row.get('nearby_beaches', ''))

    return (
        f"Nom de l'hôtel : {name}.\n"
        f"Localisation : {location}.\n"
        f"Prix : {price} TND par nuit.\n"
        f"Note : {rating}.\n"
        f"Caractéristiques : {features}.\n"
        f"Plages à proximité : {beaches}.\n"
    )

df_hotels['text'] = df_hotels.apply(generate_text, axis=1)

# Embedding

In [ ]:
# Installation des bibliothèques nécessaires
!pip install langchain langchain-community pypdf chromadb -q
!pip install langchain_groq -q
!pip install -U langchain-huggingface -q
!pip install -U langchain-chroma -q
!pip install gradio -q

!pip install language_tool_python

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 3.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 37.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 303.4/303.4 kB 27.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.9/18.9 MB 107.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 26.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 93.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.6/101.6 kB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 112.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.9/55.9 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.9/194.9 kB 17.3 MB/s eta

In [ ]:
# Importations
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.vectorstores import Chroma
from langchain.chains import RetrievalQA
from langchain_groq import ChatGroq
from langchain.prompts import PromptTemplate
from langchain.schema import Document

import os
import gradio as gr
import json

on a choisi le e5-large-v2 au lieu de all MiniLM-v6 car il gére plus de token ( 1024 aux lieu de 384) et plus adapté pour les descriptions détaillées des hotels mais il prend beaucoup de temps

In [ ]:
# Initialize the Hugging Face embeddings model
embedding_function = HuggingFaceEmbeddings(model_name="intfloat/e5-large-v2")

# Generate embeddings for the 'text' column
hotel_embeddings = embedding_function.embed_documents(df_hotels['text'].tolist())

# Print the shape of the embeddings
print(f"Shape of the embeddings: {len(hotel_embeddings)}, {len(hotel_embeddings[0])}")

le choix des chunk est :


*   size 300 et overlap 50 : trop petit et génére des erreurs
*   size 1200 et overlap 350 : prend plus que 3 heures pour executer
*   size 1000 et overlap 300 : parfait pour les hotels sans les commentaires

In [ ]:
from langchain_community.vectorstores import Chroma
from langchain.schema import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter

# Split the text into chunks
splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=300)
documents = [Document(page_content=t) for t in df_hotels['text'].tolist()]
split_docs = splitter.split_documents(documents)

# Persisted vectorstore
persist_directory = "chroma_db"


if os.path.exists(persist_directory):
    print("Chargement de la base Chroma existante...")
    vectorstore = Chroma(persist_directory=persist_directory, embedding_function=embedding_function)
else:
    print("Création d'une nouvelle base Chroma...")
    vectorstore = Chroma.from_documents(
    documents=split_docs,
    embedding=embedding_function,
    persist_directory=persist_directory,
    collection_name="hotels",
    )
vectorstore.persist()

# Now reload the persisted vectorstore
vectorstore = Chroma(
    embedding_function=embedding_function,
    persist_directory=persist_directory,
    collection_name="hotels"
)


In [ ]:
llm = ChatGroq(model="llama-3.3-70b-versatile", api_key="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")

prompt_template = PromptTemplate(
   template=(
        "tu es un assistant expert en tourisme et hôtels en particulier en tunisie"
        #resumer moi l'hotel par les avis
        "Répondez avec des informations précises et pertinentes en vous basant uniquement sur le contexte fourni.\n\n"
        "Contexte: {context}\n"
        "Question du client: {question}\n\n"
        "Réponse détaillée:"
   ),
   input_variables=["context", "question"]
)

qa_chain = RetrievalQA.from_chain_type(
   llm=llm,
   chain_type="stuff",
   retriever=vectorstore.as_retriever(),
   return_source_documents=True,
   chain_type_kwargs={"prompt": prompt_template}
)

In [ ]:
def chatbot(query):
    result = qa_chain({"query": query})
    return result['result']

iface = gr.Interface(
    fn=chatbot,
    inputs=gr.Textbox(lines=2, placeholder="Enter your question here..."),
    outputs="text",
    title="Hotel RAG Chatbot",
)

iface.launch()